# Task 1: Create a Prescription Parser using CRF
This task tests your ability to build a Doctor Prescription Parser with the help of CRF model

Your job is to build a Prescription Parser that takes a prescription (sentence) as an input and find / label the words in that sentence with one of the already pre-defined labels

### Problem: SEQUENCE PREDICTION - Label words in a sentence
#### Input : Doctor Prescription in the form of a sentence split into tokens
- Ex: Take 2 tablets once a day for 10 days

#### Output : FHIR Labels
- ('Take', 'Method')
- ('2', 'Qty')
- ('tablets', 'Form')
- ('once', 'Frequency')
- ('a', 'Period')
- ('day', 'PeriodUnit')
- ('for', 'FOR')
- ('10', 'Duration')
- ('days', 'DurationUnit')

### Major Steps
- Install necessary library
- Import the libraries
- Create training data with labels
    - Split the sentence into tokens
    - Compute POS tags
    - Create triples
- Extract features
- Split the data into training and testing set
- Create CRF model
- Save the CRF model
- Load the CRF model
- Predict on test data
- Accuracy

#### Install necesaary library

In [1]:
!pip install textacy
!python -m spacy download en_core_web_sm
!pip install sklearn_crfsuite

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
import spacy
import textacy
from textacy.extract import keyterms

#### Import the necessary libraries

In [7]:
from nltk.tag import pos_tag
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF, metrics
import pycrfsuite

### Input data (GIVEN)
#### Creating the inputs to the ML model in the following form:
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [8]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [9]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s):
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [11]:
def tuples_maker(inp, out):

#     inp = [input_sigs[0]+i for i in input_sigs[1:]][0]

#     out = []

#     for i in output_labels:
#         out.extend(i)

#     sample_data = [(i, j) for i, j in zip(inp, out)]

#     return sample_data
    return [list(zip(i[0], i[1])) for i in zip(inp, out)]

tuples_maker(input_sigs, output_labels)[-4:]

[[('after', 'AFTER'), ('food', 'FOOD')],
 [('for', 'FOR'), ('20', 'Duration'), ('days', 'DurationUnit')],
 [('for', 'FOR'), ('twenty', 'Duration'), ('days', 'DurationUnit')],
 [('with', 'WITH'), ('meals', 'FOOD')]]

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output:
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [13]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [16]:
def triples_maker(whole_data):

    # for i in tuples_maker(input_sigs, output_labels)[-1:]:
    #     for x in i:
    # #         print(pos_tag(x))

    # [list(zip(pos_tag(x[0]), x[1])) for x in data[:5]]
    # for i in tuples_maker(input_sigs, output_labels)[0]:
    # #     print(i)
    #     tag = pos_tag(i)
    #     for
    #     print(list(zip(tag[0],tag[1])))
    # [pos_tag(i) for i in [x for x in data[:5]][0]]
    # [y[0]+(y[1][0],) for y in [pos_tag(i) for i in [x for x in data[:5]][0]]]

    sample_data = [[y[0]+(y[1][0],) for y in [pos_tag(i) for i in [x for x in k]]] for k in whole_data]
    return sample_data

data = tuples_maker(input_sigs, output_labels)

triples_maker(data)[:4]

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'NN', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'NN', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'NN', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')]]

In [18]:
data = tuples_maker(input_sigs, output_labels)
sample_data = triples_maker(data) # Pass 'data' instead of 'whole_data'
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'NN', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'NN', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'NN', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
- SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag
#### Feel free to include more features

In [19]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

### Running the feature extractor on the training data
- Feature extraction
- Train-test-split

In [20]:
# function for extracting features in doc
def get_features(doc):
    return [token_to_features(doc,i) for i in range(len(doc))]

# function for generating list of labels for each doc
def get_labels(doc):
    return [label for (token, postag, label) in doc]

sample_data = triples_maker(data)

X = [get_features(doc) for doc in sample_data]
y = [get_labels(doc) for doc in sample_data]

X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.2)

### Training the CRF model with the features extracted using the feature extractor method

In [87]:
trainer = pycrfsuite.Trainer()
# Submit training data to the trainer
for feat, targ in zip(X,y):
    trainer.append(feat, targ)

# Set the parameters of the model
trainer.set_params({
    'max_iterations':1000,
    'c1':0.1,
    'c2':0.01
})

# Providing a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('prescription_parser.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1006
Seconds required: 0.022

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 733.336145
Feature norm: 1.000000
Error norm: 148.779250
Active features: 995
Line search trials: 1
Line search step: 0.005180
Seconds required for this iteration: 0.002

***** Iteration #2 *****
Loss: 458.122515
Feature norm: 5.707991
Error norm: 183.583861
Active features: 981
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.001

***** Iteration #3 *****
Loss: 333.593011
Feature norm: 7.023863
Error norm: 87.711996
Active features: 975
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.

### Predicting the test data with the built model

In [88]:
#loading the model
tagger = pycrfsuite.Tagger()
tagger.open('prescription_parser.model')

In [89]:
test_sentence =  '2 tabs every 4 hours'
# do work tokenization, lowercase()
processed_sentence = [w.lower() for w in test_sentence.split()]
# get pos_tags -> [[('2','CD'),('tabs','NN'),('every','DT'),('4','CD'),('hours','NN')]]
tagged_sentence = pos_tag(processed_sentence)
# put dummy placeholder for label
features = []
for tpls in tagged_sentence:
    tpls += (tpls[1],)
    features.append(tpls)
test_sentence = [features]
# labeled_sentence = triples_maker(list(tagged_sentence))
# test = [[('2','CD','CD'),('tabs','NN','NN'),('every','DT','DT'),('4','CD','CD'),('hours','NN','NN')]]
test_sentence

[[('2', 'CD', 'CD'),
  ('tabs', 'NNS', 'NNS'),
  ('every', 'DT', 'DT'),
  ('4', 'CD', 'CD'),
  ('hours', 'NNS', 'NNS')]]

In [90]:

x_feat = [get_features(doc) for doc in test_sentence]

In [91]:
predictions = [tagger.tag(data) for data in x_feat]

In [92]:
predictions

[['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]

### Putting all the prediction logic inside a predict method

In [93]:
def predict(sig):
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig. A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    >>> predict('2 tabs every 4 hours')
    [['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]
    >>> predict('2 tabs with food')
    [['Qty', 'Form', 'WITH', 'FOOD']]
    >>> predict('2 tabs qid x 30 days')
    [['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
    """
    sentence = [w.lower() for w in sig.split()] # word tokenization
    pos = pos_tag(sentence) # generating pos tags

    # adding dummy placeholer for the label
    features = []
    for tpls in pos:
        tpls += (tpls[1],)
        features.append(tpls)
    test_sentence = [features]

    x_feat = [get_features(doc) for doc in test_sentence]
    predictions = [tagger.tag(data) for data in x_feat]
    print(sig)
    return predictions

### Sample predictions

In [95]:
predictions = predict("take 2 tabs every 6 hours x 10 days")
print(predictions)

take 2 tabs every 6 hours x 10 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [96]:
predictions = predict("take 2 tabs every 6 hours x 10 days")
print(predictions)

take 2 tabs every 6 hours x 10 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [97]:
predictions = predict("2 capsu for 10 day at bed")
print(predictions)

2 capsu for 10 day at bed
[['Qty', 'Form', 'FOR', 'Duration', 'PeriodUnit', 'AT', 'WHEN']]


In [98]:
predictions = predict("2 capsu for 10 day at bed")
print(predictions)

2 capsu for 10 day at bed
[['Qty', 'Form', 'FOR', 'Duration', 'PeriodUnit', 'AT', 'WHEN']]


In [100]:
predictions = predict("2 capsules for 10 days at bed")
print(predictions)


2 capsules for 10 days at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [101]:
predictions = predict("2 capsules for 10 days at bed")
print(predictions)

2 capsules for 10 days at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [102]:
predictions = predict("5 days 2 tabs at bed")
print(predictions)

5 days 2 tabs at bed
[['Duration', 'DurationUnit', 'Qty', 'Form', 'AT', 'WHEN']]


In [103]:
predictions = predict("3 tabs qid x 10 weeks")
print(predictions)

3 tabs qid x 10 weeks
[['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [105]:
predictions = predict("x 30 days")
print(predictions)

x 30 days
[['FOR', 'Duration', 'DurationUnit']]


In [106]:
predictions = predict("x 20 months")
print(predictions)

x 20 months
[['FOR', 'Duration', 'DurationUnit']]


In [107]:
predictions = predict("take 2 tabs po tid for 10 days")
print(predictions)

take 2 tabs po tid for 10 days
[['Method', 'Qty', 'Form', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit']]


In [108]:
predictions = predict("take 2 capsules po every 6 hours")
print(predictions)

take 2 capsules po every 6 hours
[['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit']]


In [109]:
predictions = predict("inject 2 units pu tid")
print(predictions)

inject 2 units pu tid
[['Method', 'Qty', 'Form', 'Frequency', 'TID']]


In [110]:
predictions = predict("swallow 3 caps tid by mouth")
print(predictions)

swallow 3 caps tid by mouth
[['Method', 'Qty', 'Form', 'TID', 'BY', 'PO']]


In [111]:
predictions = predict("inject 3 units orally")
print(predictions)

inject 3 units orally
[['Method', 'Qty', 'Form', 'PO']]


In [112]:
predictions = predict("orally take 3 tabs tid")
print(predictions)

orally take 3 tabs tid
[['PO', 'Method', 'Qty', 'Form', 'TID']]


In [113]:
predictions = predict("by mouth take three caps")
print(predictions)

by mouth take three caps
[['BY', 'PO', 'Method', 'Qty', 'Form']]


In [114]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")
print(predictions)

take 3 tabs orally three times a day for 10 days at bedtime
[['Method', 'Qty', 'Form', 'PO', 'Qty', 'TIMES', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [115]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")
print(predictions)

take 3 tabs orally three times a day for 10 days at bedtime
[['Method', 'Qty', 'Form', 'PO', 'Qty', 'TIMES', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [116]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")
print(predictions)

take 3 tabs orally bid for 10 days at bedtime
[['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [117]:
predictions = predict("take 3 tabs bid orally at bed")
print(predictions)

take 3 tabs bid orally at bed
[['Method', 'Qty', 'Form', 'BID', 'PO', 'AT', 'WHEN']]


In [118]:
predictions = predict("take 10 capsules by mouth qid")
print(predictions)

take 10 capsules by mouth qid
[['Method', 'Qty', 'Form', 'BY', 'PO', 'QID']]


In [119]:
predictions = predict("inject 10 units orally qid x 3 months")
print(predictions)

inject 10 units orally qid x 3 months
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [120]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")
print(predictions)

please take 2 tablets per day for a month in the morning and evening each day
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [121]:
prediction = predict("Amoxcicillin QID 30 tablets")
print(predictions)

Amoxcicillin QID 30 tablets
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [122]:
prediction = predict("take 3 tabs TID for 90 days with food")
print(predictions)

take 3 tabs TID for 90 days with food
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [123]:
prediction = predict("with food take 3 tablets per day for 90 days")
print(predictions)

with food take 3 tablets per day for 90 days
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [61]:
prediction = predict("with food take 3 tablets per week for 90 weeks")
print(predictions)


with food take 3 tablets per week for 90 weeks


In [124]:
prediction = predict("take 2-4 tabs")
print(predictions)

take 2-4 tabs
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [125]:
prediction = predict("take 2 to 4 tabs")
print(predictions)

take 2 to 4 tabs
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [126]:
prediction = predict("take two to four tabs")
print(predictions)

take two to four tabs
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [127]:
prediction = predict("take 2-4 tabs for 8 to 9 days")
print(predictions)

take 2-4 tabs for 8 to 9 days
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [128]:
prediction = predict("take 20 tabs every 6 to 8 days")
print(predictions)

take 20 tabs every 6 to 8 days
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [129]:
prediction = predict("take 2 tabs every 4 to 6 days")
print(predictions)

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [130]:
prediction = predict("take 2 tabs every 2 to 10 weeks")
print(predictions)

take 2 tabs every 2 to 10 weeks
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [131]:
prediction = predict("take 2 tabs every 4 to 6 days")
print(predictions)

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [132]:
prediction = predict("take 2 tabs every 2 to 10 months")
print(predictions)

take 2 tabs every 2 to 10 months
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [133]:
prediction = predict("every 60 mins")
print(predictions)

every 60 mins
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [134]:
prediction = predict("every 10 mins")
print(predictions)

every 10 mins
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [135]:
prediction = predict("every two to four months")
print(predictions)

every two to four months
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [136]:
prediction = predict("take 2 tabs every 3 to 4 days")
print(predictions)

take 2 tabs every 3 to 4 days
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [137]:
prediction = predict("every 3 to 4 days take 20 tabs")
print(predictions)

every 3 to 4 days take 20 tabs
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [76]:
prediction = predict("once in every 3 days take 3 tabs")
print(predictions)


once in every 3 days take 3 tabs


In [138]:
prediction = predict("take 3 tabs once in every 3 days")
print(predictions)

take 3 tabs once in every 3 days
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [139]:
prediction = predict("orally take 20 tabs every 4-6 weeks")
print(predictions)

orally take 20 tabs every 4-6 weeks
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [140]:
prediction = predict("10 tabs x 2 days")
print(predictions)

10 tabs x 2 days
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [141]:
prediction = predict("3 capsule x 15 days")
print(predictions)

3 capsule x 15 days
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [142]:
prediction = predict("10 tabs")
print(predictions)

10 tabs
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [143]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")
print(predictions)

please take 2 tablets per day for a month in the morning and evening each day
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]
